In [3]:
!pip install --upgrade --force-reinstall --no-cache-dir "langchain>=1.0.0" "langchain-google-genai>=1.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 217.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 125.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 184.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 355.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 168.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.4/462.4 kB 331.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 216.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.7/173.7 kB 301.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.6/222.6 kB 345.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 259.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import pandas as pd
import re
import os
import io
import contextlib
from pathlib import Path
from google.colab import userdata

# --- Imports for Google Gemini ---
from langchain_google_genai import ChatGoogleGenerativeAI

# --- Securely load your Google API key ---
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print(" Google API Key loaded successfully.")
except Exception as e:
    print(f" Error loading GOOGLE_API_KEY. Make sure you have set it in Colab Secrets (🔑). Error: {e}")
    GOOGLE_API_KEY = None

 Google API Key loaded successfully.


In [2]:
def normalize_name(name):
    if not isinstance(name, str): return ""
    name = name.lower()
    name = re.sub(r'[^a-z0-9\s]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    name = name.replace(' and ', ' ').replace(' & ', ' ')
    return name


def load_all_data(base_path):
    base_path = Path(base_path)
    dataframes = {}

    try:
        fp = base_path / "APY.csv"
        df_crops = pd.read_csv(fp)
        df_crops.columns = [col.strip() for col in df_crops.columns]
        df_crops['Production'] = pd.to_numeric(df_crops['Production'], errors='coerce').fillna(0).astype('int64')

        df_crops['State_norm'] = df_crops['State'].apply(normalize_name)
        df_crops['District_norm'] = df_crops['District'].apply(normalize_name)
        df_crops = df_crops.drop(columns=['State', 'District'])
        df_crops = df_crops.rename(columns={'State_norm': 'State', 'District_norm': 'District'})

        df_crops.name = "Crop_Production (APY.csv)"
        dataframes['df_crops'] = df_crops
        print(" Loaded Crop_Production (APY.csv) as 'df_crops'")
    except Exception as e: print(f" Error loading Crop_Production: {e}")

    try:
        fp = base_path / "sub-division_rainfall_act_dep_1901-2015.csv"
        df_rainfall_ts = pd.read_csv(fp)
        df_rainfall_ts = df_rainfall_ts[df_rainfall_ts['Parameter'] == 'Actual'].copy()

        df_rainfall_ts['Subdivision_norm'] = df_rainfall_ts['SUBDIVISION'].apply(normalize_name)
        # Drop the original SUBDIVISION and the problematic Parameter column
        df_rainfall_ts = df_rainfall_ts.drop(columns=['SUBDIVISION', 'Parameter'])
        df_rainfall_ts = df_rainfall_ts.rename(columns={'Subdivision_norm': 'SUBDIVISION'})

        df_rainfall_ts.name = "Rainfall_TimeSeries (sub-division_rainfall_act_dep_1901-2015.csv)"
        dataframes['df_rainfall_ts'] = df_rainfall_ts
        print(" Loaded Rainfall_TimeSeries as 'df_rainfall_ts'")
    except Exception as e: print(f" Error loading Rainfall_TimeSeries: {e}")

    try:
        fp = base_path / "district _rainfall.csv"
        df_rainfall_dist = pd.read_csv(fp)

        df_rainfall_dist['State_norm'] = df_rainfall_dist['STATE/UT'].apply(normalize_name)
        df_rainfall_dist['District_norm'] = df_rainfall_dist['DISTRICT'].apply(normalize_name)
        df_rainfall_dist = df_rainfall_dist.drop(columns=['STATE/UT', 'DISTRICT'])
        df_rainfall_dist = df_rainfall_dist.rename(columns={'State_norm': 'STATE/UT', 'District_norm': 'DISTRICT'})

        df_rainfall_dist.name = "Rainfall_District_Normal (district _rainfall.csv)"
        dataframes['df_rainfall_district'] = df_rainfall_dist
        print(" Loaded Rainfall_District_Normal as 'df_rainfall_district'")
    except Exception as e: print(f" Error loading Rainfall_District_Normal: {e}")

    try:
        fp = base_path / "TEMP_ANNUAL_SEASONAL_MEAN.csv"
        df_temp_mean = pd.read_csv(fp)
        df_temp_mean.name = "Temperature_Mean_TimeSeries (TEMP_ANNUAL_SEASONAL_MEAN.csv)"
        dataframes['df_temp_mean'] = df_temp_mean
        print(" Loaded Temperature_Mean_TimeSeries as 'df_temp_mean'")
    except Exception as e: print(f" Error loading Temperature_Mean_TimeSeries: {e}")

    print("\n--- Data Loading Complete ---")
    return dataframes

DATA_PATH = "/content/"
dataframes = load_all_data(DATA_PATH)

 Loaded Crop_Production (APY.csv) as 'df_crops'
 Loaded Rainfall_TimeSeries as 'df_rainfall_ts'
 Loaded Rainfall_District_Normal as 'df_rainfall_district'
 Loaded Temperature_Mean_TimeSeries as 'df_temp_mean'

--- Data Loading Complete ---


In [3]:
# --- 1. Set up the LLM ---
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0)

# --- 2. Create the "Tool": a safe Python code executor ---
def execute_python_code(code: str, df_dict: dict):
    try:
        buffer = io.StringIO()
        with contextlib.redirect_stdout(buffer):
            exec(code, {}, df_dict)
        output = buffer.getvalue()
        if not output: output = "Code executed successfully (no print output)."
        return output
    except Exception as e:
        return f"Error executing code: {str(e)}"

# --- 3. Get Dataframe Info for the Prompt ---
def get_dataframe_schemas(df_dict: dict):
    schemas = ""
    for name, df in df_dict.items():
        schemas += f"\n--- Dataframe: '{name}' (Original File: {df.name}) ---\n"
        buffer = io.StringIO()
        df.info(buf=buffer)
        schemas += buffer.getvalue()
        schemas += "\n"
    return schemas

# --- 4. The Master Prompt ---
AGENT_PROMPT_TEMPLATE = r"""
You are 'Samarth', an expert policy and agriculture data analyst.
Your task is to answer questions by generating and executing Python code on pandas dataframes.

You must follow a strict "Thought -> Action -> Observation -> Final Answer" loop.

**RULES:**
1.  **Thought:** Briefly explain your plan.
2.  **Action:** Provide *one and only one* Python code block to execute. This MUST be in a code block `Action:\n```python\n(your code)\n````.
3.  **Observation:** After your code is run, you will be given the output or error.
4.  **Repeat:** Repeat Thought/Action until you have the data.
5.  **Final Answer:** When you have all the information, you **MUST** provide the final, clean, natural language answer, and it **MUST** be prefaced with the tag `Final Answer:`.

**IMPORTANT CONSTRAINTS:**
* You can only use the dataframes provided below: {df_names}
* The pandas library is available as `pd`.

**CRITICAL DATA RULES:**
* **DATA IS CLEAN:** All string columns (`State`, `District`, `SUBDIVISION`) are **already normalized and in lowercase**.
* **USE LOWERCASE:** You **MUST** use lowercase strings when filtering.
    * Example: `df_crops['State'] == 'maharashtra'` (CORRECT)
    * Example: `df_crops['State'] == 'MAHARASHTRA'` (INCORRECT - will fail)
* **DATA CLEANING (CROPS):** When finding top crops, you **MUST** exclude any rows where the 'Crop' name is 'Total Foodgrain' or contains the word 'Total'.
* **RAINFALL LOGIC:** To find average rainfall for a region, you must calculate the simple mean of the 'ANNUAL' column for that region.
    * Example: `region_data['ANNUAL'].mean()` (CORRECT)
    * Example: `region_data.groupby('YEAR')['ANNUAL'].sum().mean()` (INCORRECT)
* **CITATIONS:** Your `Final Answer:` **MUST** cite the original source filename.
* **Do not** mention the *variable names* (like 'df_crops') in your final answer.

**AVAILABLE DATAFRAMES:**
{df_schemas}

**START OF CONVERSATION:**

Question: {user_question}
Thought:
"""

# --- 5. The Agent Loop Function ---
def run_agent(user_question: str, df_dict: dict):
    print(f"--- Agent processing question: {user_question} ---")
    df_names = list(df_dict.keys())
    df_schemas = get_dataframe_schemas(df_dict)

    prompt = AGENT_PROMPT_TEMPLATE.format(
        df_names=df_names,
        df_schemas=df_schemas,
        user_question=user_question
    )

    conversation_history = [prompt]

    for _ in range(5):
        print(f"--- Sending prompt to LLM (Step {_ + 1}) ---")

        full_prompt = "\n".join(conversation_history)
        ai_message = llm.invoke(full_prompt)
        llm_content = ai_message.content

        if isinstance(llm_content, list):
            llm_response_str = "\n".join(str(part) for part in llm_content)
        else:
            llm_response_str = str(llm_content)

        conversation_history.append(llm_response_str)
        print(f"--- LLM Response: ---\n{llm_response_str}\n")

        if "Final Answer:" in llm_response_str:
            final_answer = llm_response_str.split("Final Answer:")[-1].strip()
            print(f"--- Final Answer Found ---")
            return final_answer

        code_matches = re.findall(r"Action:\n```python\n(.*?)\n```", llm_response_str, re.DOTALL)

        if code_matches:
            code_to_run = code_matches[-1].strip()
            print(f"--- Executing Code: ---\n{code_to_run}\n")

            exec_context = {**df_dict, "pd": pd}
            code_result = execute_python_code(code_to_run, exec_context)

            print(f"--- Code Result (Observation): ---\n{code_result}\n")
            conversation_history.append(f"Observation: {code_result}")
        else:
            print("--- LLM did not provide a valid Action or Final Answer. Asking to reformat. ---")
            conversation_history.append("Observation: Your response was not in the correct format. Please provide an `Action:` block or a `Final Answer:` tag.")

    return "Sorry, I couldn't find an answer in 5 steps."

print("\n--- Manual Agent is Ready ---")


--- Manual Agent is Ready ---


In [4]:
# --- Test Query 1 (Sample Question 1) ---
question1 = """
Compare the average annual rainfall in Maharashtra and Karnataka for the last 5 available years (2011-2015).
In parallel, list the top 3 most produced crops (by production volume) in each of those states during the same 2011-2015 period, citing all data sources.
"""
response1 = run_agent(question1, dataframes)
print(f"\n====================\nFinal Answer 1:\n{response1}\n====================\n")


# --- Test Query 2 (Sample Question 2) ---
question2 = """
Identify the district in Maharashtra with the highest production of Rice in the most recent year available (2015).
And compare that with the district with the lowest production of Rice in Karnataka in 2015.
"""
response2 = run_agent(question2, dataframes)
print(f"\n====================\nFinal Answer 2:\n{response2}\n====================\n")

--- Agent processing question: 
Compare the average annual rainfall in Maharashtra and Karnataka for the last 5 available years (2011-2015).
In parallel, list the top 3 most produced crops (by production volume) in each of those states during the same 2011-2015 period, citing all data sources.
 ---
--- Sending prompt to LLM (Step 1) ---
--- LLM Response: ---
Thought:
First, I need to determine the average annual rainfall for Maharashtra and Karnataka from 2011 to 2015. The `df_rainfall_ts` dataframe is suitable for this. I'll start by identifying the subdivisions within Maharashtra and Karnataka. Then, I'll filter the dataframe for these subdivisions and the specified years (2011-2015) and calculate the mean of the 'ANNUAL' rainfall column for each state.

To begin, I will list all unique subdivisions to identify the relevant ones for Maharashtra and Karnataka.Action:
```python
print(df_rainfall_ts['SUBDIVISION'].unique())
```
Observation:
```
['andaman & nicobar islands' 'arunachal pr

In [5]:
%%writefile app.py

import streamlit as st
import os
import pandas as pd
import re
import io
import contextlib
from pathlib import Path
from io import StringIO
from langchain_google_genai import ChatGoogleGenerativeAI

# --- 1. Data Loading Functions ---
def normalize_name(name):
    if not isinstance(name, str): return ""
    name = name.lower()
    name = re.sub(r'[^a-z0-9\s]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    name = name.replace(' and ', ' ').replace(' & ', ' ')
    return name

@st.cache_resource
def load_all_data(base_path):
    base_path = Path(base_path)
    dataframes = {}
    schemas = ""

    def get_schema(df, name, filename):
        schema_str = f"\n--- Dataframe: '{name}' (Original File: {filename}) ---\n"
        buffer = io.StringIO()
        df.info(buf=buffer)
        schema_str += buffer.getvalue()
        return schema_str + "\n"

    try:
        fp = base_path / "APY.csv"
        df_crops = pd.read_csv(fp)
        df_crops.columns = [col.strip() for col in df_crops.columns]
        df_crops['Production'] = pd.to_numeric(df_crops['Production'], errors='coerce').fillna(0).astype('int64')
        df_crops['State_norm'] = df_crops['State'].apply(normalize_name)
        df_crops['District_norm'] = df_crops['District'].apply(normalize_name)
        df_crops = df_crops.drop(columns=['State', 'District'])
        df_crops = df_crops.rename(columns={'State_norm': 'State', 'District_norm': 'District'})
        df_crops.name = "APY.csv"
        dataframes['df_crops'] = df_crops
        schemas += get_schema(df_crops, 'df_crops', 'APY.csv')
    except Exception as e: st.error(f"Failed to load APY.csv: {e}")

    try:
        fp = base_path / "sub-division_rainfall_act_dep_1901-2015.csv"
        df_rainfall_ts = pd.read_csv(fp)
        df_rainfall_ts = df_rainfall_ts[df_rainfall_ts['Parameter'] == 'Actual'].copy()
        df_rainfall_ts['Subdivision_norm'] = df_rainfall_ts['SUBDIVISION'].apply(normalize_name)
        df_rainfall_ts = df_rainfall_ts.drop(columns=['SUBDIVISION', 'Parameter'])
        df_rainfall_ts = df_rainfall_ts.rename(columns={'Subdivision_norm': 'SUBDIVISION'})
        df_rainfall_ts.name = "sub-division_rainfall_act_dep_1901-2015.csv"
        dataframes['df_rainfall_ts'] = df_rainfall_ts
        schemas += get_schema(df_rainfall_ts, 'df_rainfall_ts', 'sub-division_rainfall_act_dep_1901-2015.csv')
    except Exception as e: st.error(f"Failed to load sub-division_rainfall_act_dep_1901-2015.csv: {e}")

    try:
        fp = base_path / "district _rainfall.csv"
        df_rainfall_dist = pd.read_csv(fp)
        df_rainfall_dist['State_norm'] = df_rainfall_dist['STATE/UT'].apply(normalize_name)
        df_rainfall_dist['District_norm'] = df_rainfall_dist['DISTRICT'].apply(normalize_name)
        df_rainfall_dist = df_rainfall_dist.drop(columns=['STATE/UT', 'DISTRICT'])
        df_rainfall_dist = df_rainfall_dist.rename(columns={'State_norm': 'STATE/UT', 'District_norm': 'DISTRICT'})
        df_rainfall_dist.name = "district _rainfall.csv"
        dataframes['df_rainfall_district'] = df_rainfall_dist
        schemas += get_schema(df_rainfall_dist, 'df_rainfall_district', 'district _rainfall.csv')
    except Exception as e: st.error(f"Failed to load district _rainfall.csv: {e}")

    try:
        fp = base_path / "TEMP_ANNUAL_SEASONAL_MEAN.csv"
        df_temp_mean = pd.read_csv(fp)
        df_temp_mean.name = "TEMP_ANNUAL_SEASONAL_MEAN.csv"
        dataframes['df_temp_mean'] = df_temp_mean
        schemas += get_schema(df_temp_mean, 'df_temp_mean', 'TEMP_ANNUAL_SEASONAL_MEAN.csv')
    except Exception as e: st.error(f"Failed to load TEMP_ANNUAL_SEASONAL_MEAN.csv: {e}")

    print("Data loaded successfully for Streamlit app.")
    return dataframes, schemas


# --- 2. Manual Agent Functions ---

@st.cache_resource
def get_llm():
    """Initializes the LLM."""
    try:
        if "GOOGLE_API_KEY" not in os.environ:
            st.error("API Key not found. Please set it in the Colab notebook.")
            return None
    except Exception as e:
        st.error(f"Error checking API key: {e}")
        return None

    llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0)
    return llm

def execute_python_code(code: str, df_dict: dict):
    try:
        buffer = io.StringIO()
        with contextlib.redirect_stdout(buffer):
            exec(code, {}, df_dict)
        output = buffer.getvalue()
        if not output: output = "Code executed successfully (no print output)."
        return output
    except Exception as e:
        return f"Error executing code: {str(e)}"

def parse_stdout_to_dataframe(stdout_str):
    """Tries to parse the print() output into a DataFrame."""
    try:
        data = StringIO(stdout_str)
        df = pd.read_csv(data, sep=r'\s{2,}', engine='python', index_col=False)

        if 'Unnamed: 0' in df.columns:
            df = df.drop(columns=['Unnamed: 0'])

        if all(col.startswith('Unnamed:') for col in df.columns):
            return None

        return df
    except Exception as e:
        print(f"Failed to parse stdout to DataFrame: {e}")
        return None

AGENT_PROMPT_TEMPLATE = r"""
You are 'Samarth', an expert policy and agriculture data analyst.
Your task is to answer questions by generating and executing Python code on pandas dataframes.
You must follow a strict "Thought -> Action -> Observation -> Final Answer" loop.
**RULES:**
1.  **Thought:** Briefly explain your plan.
2.  **Action:** Provide *one and only one* Python code block to execute. This MUST be in a code block `Action:\n```python\n(your code)\n````.
3.  **Observation:** After your code is run, you will be given the output or error.
4.  **Repeat:** Repeat Thought/Action until you have the data.
5.  **Final Answer:** When you have all the information, you **MUST** provide the final, clean, natural language answer, and it **MUST** be prefaced with the tag `Final Answer:`.
**IMPORTANT CONSTRAINTS:**
* You can only use the dataframes provided below: {df_names}
* The pandas library is available as `pd`.

**CRITICAL DATA RULES:**
* **DATA IS CLEAN:** All string columns (`State`, `District`, `SUBDIVISION`) are **already normalized and in lowercase**.
* **USE LOWERCASE:** You **MUST** use lowercase strings when filtering.
    * Example: `df_crops['State'] == 'maharashtra'` (CORRECT)
    * Example: `df_crops['State'] == 'MAHARASHTRA'` (INCORRECT - will fail)
* **DATA CLEANING (CROPS):** When finding top crops, you **MUST** exclude any rows where the 'Crop' name is 'Total Foodgrain' or contains the word 'Total'.
* **RAINFALL LOGIC:** To find average rainfall for a region, you must calculate the simple mean of the 'ANNUAL' column for that region.
    * Example: `region_data['ANNUAL'].mean()` (CORRECT)
    * Example: `region_data.groupby('YEAR')['ANNUAL'].sum().mean()` (INCORRECT)
* **CITATIONS:** Your `Final Answer:` **MUST** cite the original source filename.
* **Do not** mention the *variable names* (like 'df_crops') in your final answer.

**AVAILABLE DATAFRAMES:**
{df_schemas}
**START OF CONVERSATION:**
Question: {user_question}
Thought:
"""

def run_agent_for_app(user_question: str, df_dict: dict, schemas: str, llm):
    df_names = list(df_dict.keys())

    prompt = AGENT_PROMPT_TEMPLATE.format(
        df_names=df_names,
        df_schemas=schemas,
        user_question=user_question
    )

    conversation_history = [prompt]

    for _ in range(5):
        yield {"type": "spinner", "content": "Thinking..."}

        full_prompt = "\n".join(conversation_history)
        ai_message = llm.invoke(full_prompt)
        llm_content = ai_message.content

        if isinstance(llm_content, list):
            llm_response_str = "\n".join(str(part) for part in llm_content)
        else:
            llm_response_str = str(llm_content)

        conversation_history.append(llm_response_str)

        if "Final Answer:" in llm_response_str:
            final_answer = llm_response_str.split("Final Answer:")[-1].strip()
            yield {"type": "final_answer", "content": final_answer}
            return

        code_matches = re.findall(r"Action:\n```python\n(.*?)\n```", llm_response_str, re.DOTALL)

        if code_matches:
            code_to_run = code_matches[-1].strip()

            exec_context = {**df_dict, "pd": pd}
            code_result = execute_python_code(code_to_run, exec_context)

            yield {"type": "observation", "content": code_result}

            conversation_history.append(f"Observation: {code_result}")
        else:
            yield {"type": "error", "content": "The agent did not provide a valid Action or Final Answer. Please rephrase your question."}
            return

    yield {"type": "error", "content": "Sorry, I couldn't find an answer in 5 steps."}


# --- 3. Streamlit App UI ---
st.set_page_config(page_title="Project Samarth Q&A", layout="wide")
st.title("Project Samarth: Intelligent Q&A")
st.markdown("Query agricultural and climate data.")

DATA_PATH = "/content/"
dataframes, schemas = load_all_data(DATA_PATH)
llm = get_llm()

if dataframes:
    st.markdown("---")
    st.subheader("Data Dashboard")

    try:
        col1, col2 = st.columns(2)

        with col1:
            # Chart 1: Top 5 Crops (All Time)
            st.markdown("##### Top 5 Crops (All-Time Production)")
            df_crops = dataframes['df_crops']
            top_5_crops = df_crops[
                (~df_crops['Crop'].str.contains('total', na=False)) & # Use lowercase
                (df_crops['Crop'] != 'Total Foodgrain') # This specific one is still proper case
            ].groupby('Crop')['Production'].sum().nlargest(5)
            st.bar_chart(top_5_crops)

        with col2:
            # Chart 2: All-India Annual Rainfall Trend
            st.markdown("##### All-India Rainfall Trend (1901-2015)")
            df_rain = dataframes['df_rainfall_ts'].copy()
            df_rain['YEAR'] = pd.to_numeric(df_rain['YEAR'], errors='coerce')
            rain_trend = df_rain.dropna(subset=['YEAR'])
            rain_trend = rain_trend[rain_trend['YEAR'] <= 2015].groupby('YEAR')['ANNUAL'].mean()
            st.line_chart(rain_trend)

    except Exception as e:
        st.error(f"Error generating dashboard: {e}")

    with st.expander("About this Project & Available Data"):
        st.markdown("""
        This is a prototype Q&A system for **Project Samarth**. It uses a generative AI agent to answer questions by writing and executing Python code on live government datasets.

        **Available Datasets:**
        * **Crop Production (`APY.csv`):** State, District, Crop, Year, Season, Area, and Production data.
        * **Rainfall Time-Series (`sub-division_rainfall_act_dep_1901-2015.csv`):** Monthly and annual rainfall data for meteorological sub-divisions.
        * **District Rainfall (`district _rainfall.csv`):** Average (normal) rainfall for each district.
        * **Temperature (`TEMP_ANNUAL_SEASONAL_MEAN.csv`):** All-India annual mean temperature time-series.
        """)

    st.markdown("---")
    st.subheader("Live Chat Query")


if dataframes and llm:
    if "messages" not in st.session_state:
        st.session_state.messages = []

    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            for item in message["content"]:
                if item["type"] == "text":
                    st.markdown(item["content"])
                elif item["type"] == "dataframe":
                    st.dataframe(item["content"])
                elif item["type"] == "error":
                    st.error(item["content"])

    if prompt := st.chat_input("Ask a question about crop production and climate..."):
        st.session_state.messages.append({"role": "user", "content": [{"type": "text", "content": prompt}]})

        with st.chat_message("user"):
            st.markdown(prompt)

        with st.chat_message("assistant"):
            assistant_response_content = []
            current_spinner = st.spinner("Thinking...")

            with current_spinner:
                try:
                    for update in run_agent_for_app(prompt, dataframes, schemas, llm):

                        if update["type"] == "spinner":
                            continue

                        elif update["type"] == "observation":
                            parsed_df = parse_stdout_to_dataframe(update["content"])

                            if parsed_df is not None:
                                st.dataframe(parsed_df)
                                assistant_response_content.append({"type": "dataframe", "content": parsed_df})
                            else:

                                pass

                        elif update["type"] == "final_answer":
                            st.markdown(update["content"])
                            assistant_response_content.append({"type": "text", "content": update["content"]})

                        elif update["type"] == "error":
                            st.error(update["content"])
                            assistant_response_content.append(update)

                    st.session_state.messages.append({"role": "assistant", "content": assistant_response_content})

                except Exception as e:
                    error_message = f"An error occurred: {str(e)}"
                    st.error(error_message)
                    st.session_state.messages.append({"role": "assistant", "content": [{"type": "error", "content": error_message}]})
else:
    st.error("Could not load data or initialize LLM. Please check your CSV files and GOOGLE_API_KEY environment variable.")

Writing app.py


In [6]:
!pip install pyngrok streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 67.8 MB/s eta 0:00:00


In [7]:
!curl ifconfig.me

34.186.3.70

In [8]:
from google.colab import userdata
import os
from pyngrok import ngrok
import subprocess
import time

# 1. Get Google API Key
try:
    api_key = userdata.get('GOOGLE_API_KEY')
    print("Google API Key retrieved.")
except Exception as e:
    print(f"Error getting Google API Key. Please add it to Colab Secrets. Error: {e}")
    raise SystemExit()

# 2. Set Ngrok Auth Token
try:
    ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
    ngrok.set_auth_token(ngrok_token)
    print("Ngrok auth token set.")
except Exception as e:
    print(f"Error setting Ngrok token. Please add it to Colab Secrets as 'NGROK_AUTH_TOKEN'. Error: {e}")
    raise SystemExit()

# 3. Start Streamlit in the background
print("Starting Streamlit app in background...")

env = os.environ.copy()
env["GOOGLE_API_KEY"] = api_key


# Pass the modified environment to the subprocess
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'], env=env)

# Wait 5 seconds for the app to start
time.sleep(5)

# 4. Create the pyngrok tunnel
try:
    # Kill any existing tunnels
    for tunnel in ngrok.get_tunnels():
        ngrok.disconnect(tunnel.public_url)
        print(f"Closed old tunnel: {tunnel.public_url}")

    public_url = ngrok.connect(8501)
    print("----------------------------------------------------------")
    print(f"YOUR APP IS LIVE AT: {public_url}")
    print("----------------------------------------------------------")
except Exception as e:
    print(f"Error starting ngrok tunnel: {e}")

Google API Key retrieved.
Ngrok auth token set.
Starting Streamlit app in background...
----------------------------------------------------------
YOUR APP IS LIVE AT: NgrokTunnel: "https://unexpressable-independently-johna.ngrok-free.dev" -> "http://localhost:8501"
----------------------------------------------------------
